In [1]:
import numpy as np
import pandas as pd
import lmdb
import h5py
import os

from keras.preprocessing import image

Using TensorFlow backend.


In [94]:
def prepare_pascal3d(data_path, out_path):
    """Converts cropped images and orientation annotation from Pascal3D+ into h5 database.
    
    Pascal3D+ should be preprocessed first via 
        https://github.com/ShapeNet/RenderForCNN/blob/master/view_estimation/prepare_testing_data.py
        https://github.com/ShapeNet/RenderForCNN/blob/master/view_estimation/prepare_training_data.py
    
    # Arguments
        data_path: path to preprocessed data (/pathto/RenderForCNN/data/real_images/)
        
        out_path: savepath for created h5 database
        
    """
    
    classes = {0: 'aeroplane',
               1: 'bicycle',
               2: 'boat',
               3: 'bottle',
               4: 'bus',
               5: 'car',
               6: 'chair',
               7: 'diningtable',
               8: 'motorbike',
               9: 'sofa',
               10: 'train',
               11: 'tvmonitor'}
    
    train_labels_global_path = os.path.join(data_path, 'voc12train_all_gt_bbox')
    train_labels_paths = [os.path.join(train_labels_global_path, (classes[key]+'.txt')) for key in classes.keys()]
    
    val_labels_global_path = os.path.join(data_path, 'voc12val_easy_gt_bbox')
    val_labels_paths = [os.path.join(val_labels_global_path, (classes[key]+'.txt')) for key in classes.keys()]
    
    pascal_db = h5py.File(out_path, 'w')
    
    def _load_data_by_labels(lpath, class_name, data_part):
        
        print("loading labels from path : %s" % lpath)
        labels = pd.read_csv(lpath, delimiter=' ', header=None, 
                             names=['image_path', 'class_id', 'azimuth', 'elevation', 'tilt'])

        labels_np = np.asarray(labels[['class_id', 'azimuth', 'elevation', 'tilt']])

        images = np.zeros([labels.shape[0], 224, 224, 3])

        for i, img in enumerate(list(labels.image_path)):
            img = image.load_img(img, target_size=(224, 224))
            x = image.img_to_array(img)
            images[i] = x
        
        pascal_db.create_dataset(os.path.join(data_part, class_name, 'labels'), data=labels_np)
        pascal_db.create_dataset(os.path.join(data_part, class_name, 'images'), data=images)
        
        return 
    
    for cid, lpath in enumerate(train_labels_paths):
        _load_data_by_labels(lpath, classes[cid], 'train')
    
    for cid, lpath in enumerate(val_labels_paths):
        _load_data_by_labels(lpath, classes[cid], 'validation')
        
    pascal_db.close()
    
    return

In [95]:
data_path = '/home/sprokudin/RenderForCNN/data/real_images/'
db_path = 'data/pascal3d.h5'

In [98]:
prepare_pascal3d(data_path, db_path)

loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/aeroplane.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/bicycle.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/boat.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/bottle.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/bus.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/car.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/chair.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/diningtable.txt
loading labels from path : /home/sprokudin/RenderForCNN/data/real_images/voc12train_all_gt_bbox/motorbike.txt
loading labels from path : /home/s

In [47]:
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
data_file.create_dataset('group_name', data=data_matrix)
data_file.close()